# OMOP Data Exploration and Analysis with PostgreSQL

This jupyter notebook includes a example for querying and analyzing OMOP(Observational Medical Outcomes Partnership) data stored in a PostgreSQL database. The notebook covers essential steps, from establishing a connection to the database to executing SQL queries, and visualizing key insights.

### Importing required Libraries

- **psycopg2** For establishing a connection between python and a PostgreSQL database.
- **pandas** For efficient data handling, manipulation nad analysis of data.
- **sqlio** For executing SQL queries and reading the results directly into pandas dataframe.

In [1]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2 as ps

### Database connection and setup

In [2]:
conn = ps.connect(dbname="postgres",
                   user = "ohdsi_admin_user",
                   password = "admin1",
                   host = "omop-db-postgress",
                   port = "5432")

In [3]:
df = sqlio.read_sql_query("SELECT * FROM person", conn)

/tmp/ipykernel_782/1694805293.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = sqlio.read_sql_query("SELECT * FROM person", conn)


In [4]:
df.head()

,id,name,age,email
0,1,John Doe,25,john@example.com
1,2,Jane Smith,30,jane@example.com
2,3,Bob Johnson,28,bob@example.com
3,4,Alice Brown,22,alice@example.com
4,5,Charlie Davis,35,charlie@example.com
